# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 78 new papers today
          45 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/44 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.03748


extracting tarball to tmp_2210.03748...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.03754


extracting tarball to tmp_2210.03754...

...

 done.


Retrieving document from  https://arxiv.org/e-print/2210.04785


extracting tarball to tmp_2210.04785... done.
Retrieving document from  https://arxiv.org/e-print/2210.04786


extracting tarball to tmp_2210.04786...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.04721-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.04721) | **The Gaia-ESO Survey: Lithium measurements and new curves of growth**  |
|| E. Franciosini, et al. -- incl., <mark>K. Lind</mark>, <mark>G. Guiglion</mark>, <mark>E.J. Alfaro</mark>, <mark>M. Bergemann</mark>, <mark>U. Heiter</mark>, <mark>J. Lewis</mark> |
|*Appeared on*| *2022-10-11*|
|*Comments*| *15 pages, 15 figures. Accepted by Astronomy & Astrophysics*|
|**Abstract**| The Gaia-ESO Survey (GES) is a large public spectroscopic survey that was carried out using the multi-object FLAMES spectrograph at the Very Large Telescope. The survey provides accurate radial velocities, stellar parameters, and elemental abundances for ~115,000 stars in all Milky Way components. In this paper we describe the method adopted in the final data release to derive lithium equivalent widths (EWs) and abundances. Lithium EWs were measured using two different approaches for FGK and M-type stars, to account for the intrinsic differences in the spectra. For FGK stars, we fitted the lithium line using Gaussian components, while direct integration over a predefined interval was adopted for M-type stars. Care was taken to ensure continuity between the two regimes. Abundances were derived using a new set of homogeneous curves of growth that were derived specifically for GES, and which were measured on a synthetic spectral grid consistently with the way the EWs were measured. The derived abundances were validated by comparison with those measured by other analysis groups using different methods. Lithium EWs were measured for ~40,000 stars, and abundances could be derived for ~38,000 of them. The vast majority of the measures (80%) have been obtained for stars in open cluster fields. The remaining objects are stars in globular clusters, or field stars in the Milky Way disc, bulge, and halo. The GES dataset of homogeneous lithium abundances described here will be valuable for our understanding of several processes, from stellar evolution and internal mixing in stars at different evolutionary stages to Galactic evolution. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03755-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03755) | **The metallicity's fundamental dependence on both local and global  galactic quantities**  |
|| William M. Baker, et al. -- incl., <mark>Lihwai Lin</mark>, <mark>Mallory Thorp</mark> |
|*Appeared on*| *2022-10-11*|
|*Comments*| *22 pages, resubmitted to MNRAS following first referee report*|
|**Abstract**| We study the scaling relations between gas-phase metallicity, stellar mass surface density ($\Sigma _*$), star formation rate surface density ($\Sigma _{SFR}$), and molecular gas surface density ($\Sigma_{H_2}$) in local star-forming galaxies on scales of a kpc. We employ optical integral field spectroscopy from the MaNGA survey, and ALMA data for a subset of MaNGA galaxies. We use Partial Correlation Coefficients and Random Forest regression to determine the relative importance of local and global galactic properties in setting the gas-phase metallicity. We find that the local metallicity depends primarily on $\Sigma _*$ (the resolved mass-metallicity relation, rMZR), and has a secondary anti-correlation with $\Sigma _{SFR}$ (i.e. a spatially-resolved version of the `Fundamental Metallicity Relation', rFMR). We find that $\Sigma_{H_2}$ has little effect in determining the local metallicity. This result indicates that gas accretion, resulting in local metallicity dilution and local boosting of star formation, cannot be the primary origin of the rFMR. Star-formation driven, metal-loaded winds may contribute to the anti-correlation between metallicity and SFR. The local metallicity depends also on the global properties of galaxies. We find a strong dependence on the total stellar mass ($M_*$) and a weaker (inverse) dependence on the total SFR. The global metallicity scaling relations, therefore, do not simply stem out of their resolved counterparts; global properties and processes, such as the global gravitational potential well, galaxy-scale winds and global redistribution/mixing of metals, likely contribute to the local metallicity, in addition to local production and retention. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.04685-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.04685) | **The Nearest Neutron Star Candidate in a Binary Revealed by Optical  Time-domain Surveys**  |
|| <mark>Ling-Lin Zheng</mark>, et al. -- incl., <mark>Zhi-Xiang Zhang</mark>, <mark>Jia Zhang</mark>, <mark>Chun-Qian Li</mark>, <mark>Xiang-Dong Li</mark>, <mark>Zhongrui Bai</mark>, <mark>Haotong Zhang</mark>, <mark>Jifeng Liu</mark> |
|*Appeared on*| *2022-10-11*|
|*Comments*| *35 pages, 8 figures, to be submitted*|
|**Abstract**| Recent studies have revealed the global deposition on Earth of radioactive elements (e.g., $^{60}$Fe) resulting from the metal-enriched ejecta of nearby (within $\sim 100$ pc) supernova explosions. The majority of neutron stars in our Solar neighborhood remain to be discovered. Here we report the discovery of the nearest ($127.7 \pm 0.3$ pc) neutron star candidate in the single-lined spectroscopic binary LAMOST J235456.76+335625.7 (hereafter J2354). Utilizing the multi-epoch spectra and high-cadence periodic light curves, we measure the mass of the visible star ($M_{\rm vis}=0.70\pm 0.05\ M_{\odot}$) and determine the mass function of the invisible object $f(M)=0.525 \pm 0.004\ M_{\odot}$, i.e., the mass of the unseen compact object is $M_{\rm inv} \geq 1.26 \pm 0.03\ M_{\odot}$. The excess UV emission due to a hot supramassive white dwarf is absent. Hence, it is likely that J2354 harbors a neutron star. J2354 is X-ray dim (the $0.1$--$2.4$ keV luminosity $<10^{30}\ {\rm erg\ s^{-1}}$) since it is not detected in the ROSAT all-sky surveys in X-ray. One-hour exceptionally sensitive radio follow-up observations with FAST, the largest single-dish radio telescope, failed to reveal any radio pulsating signals (the potential pulse power at $1.4$ GHz is $<6.8\times 10^{23}\ {\rm erg\ s^{-1}}$). Hence, the neutron star candidate in J2354 can only be discovered via our time-resolved observations. The alternative scenario involving a nearby supramassive cold white dwarf cannot be fully excluded. Our discovery demonstrates a promising way to unveil the missing population of backyard inactive neutron stars or supramassive cold white dwarfs in binaries by exploring the optical time domain, thereby facilitating understanding of the supernovae explosion and metal-enrichment history in our Solar neighborhood. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.04786-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.04786) | **MUSE crowded field 3D spectroscopy in NGC 300 III. Characterizing  extremely faint HII regions and diffuse ionized gas**  |
|| Genoveva Micheva, et al. -- incl., <mark>Martin M. Roth</mark> |
|*Appeared on*| *2022-10-11*|
|*Comments*| *Accepted in A&A*|
|**Abstract**| There are known differences between the physical properties of HII and diffuse ionized gas (DIG), but most of the studied regions in the literature are relatively bright. We compiled a faint sample of 390 HII regions with median $\log_{10}H\alpha$=34.7 in the spiral galaxy NGC300, derived their physical properties in terms of metallicity, density, extinction, and kinematics, and performed a comparative analysis of the properties of the DIG. We used MUSE data of nine fields in NGC300, covering a galactocentric distance of zero to ~450 arcsec (~4 projected kpc), including spiral arm and inter-arm regions. We binned the data in dendrogram leaves and extracted all strong nebular emission lines. We identified HII and DIG regions and compared their electron densities, metallicity, extinction, and kinematic properties. We also tested the effectiveness of unsupervised machine-learning algorithms in distinguishing between the HII and DIG regions. The gas density in the HII and DIG regions is close to the low-density limit in all fields. The average velocity dispersion in the DIG is higher than in the HII regions, which can be explained by the DIG being 1.8 kK hotter than HII gas. The DIG manifests a lower ionization parameter than HII gas, and the DIG fractions vary between 15-77%, with strong evidence of a contribution by hot low-mass evolved stars and shocks to the DIG ionization. Most of the DIG is consistent with no extinction and an oxygen metallicity that is indistinguishable from that of the HII gas.We observe a flat metallicity profile in the central region, without a sign of a gradient. The differences between extremely faint HII and DIG regions follow the same trends and correlations as their much brighter cousins. HII and DIG are so heterogeneous, however, that the differences within each class are larger than the differences between the two classes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.04000-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.04000) | **Prospects for Constraining interacting dark energy cosmology with  gravitational-wave bright sirens detected by future SKA-era pulsar timing  arrays**  |
|| Bo Wang, <mark>Dong-Ze He</mark>, <mark>Hai-Li Li</mark>, <mark>Yi Zhang</mark> |
|*Appeared on*| *2022-10-11*|
|*Comments*| **|
|**Abstract**| Pulsar timing arrays (PTAs) have the potential to detect Nanohertz gravitational waves (GWs) that are usually generated by the individual inspiraling supermassive black hole binaries (SMBHBs) in the galactic centers. The GW signals as cosmological standard sirens can provide the absolute cosmic distances, thereby can be used to constrain the cosmological parameters. In this paper, we analyze the ability of future SKA-era PTAs to detect the existing SMBHBs candidates assuming the root mean square of timing noise $\sigma_t=20\ {\rm ns}$, and use the simulated PTA data to constrain the interacting dark energy (IDE) models with energy transfer rate $Q = \beta H\rho_c$. We find that, the future SKA-era PTAs will play an important role in constraining the IDE cosmology. Using only the mock PTA data consisting of 100 pulsars, we obtain $\sigma(H_0)=0.239\ {\rm km} \ {\rm s}^{-1} {\rm Mpc}^{-1}$ and $\sigma(\Omega_m)=0.0103$ in the I$\Lambda$CDM model, which are much better than the results from the Planck TT, TE, EE+lowE. However, the PTA data cannot provide a tight constraint on the coupling parameter $\beta$ compared with Planck, but the data combination of Planck+PTA can provide a rather tight constraint, i.e., $\sigma(\beta)=0.00232$, since the PTA data could break the parameter degeneracies inherent in CMB. In the I$w$CDM model, we obtain $\sigma(\beta)=0.00137$ and $\sigma(w)=0.0492$ from the Planck+PTA data combination. In addition, we also find that with the increase of the number of pulsars in PTA, the constraint results from the Planck+PTA will be further improved to some extent. We show that the observations of Nanohertz GWs with future SKA-era PTAs will provide a powerful tool for exploring the nature of dark energy and measuring the coupling between dark energy and dark matter. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xb2 in position 34292: invalid start byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03764-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03764) | **VLBA reveals the absence of a compact radio core in the radio  intermediate quasar J2242+0334 at z =5.9**  |
|| <mark>Yuanqi Liu</mark>, et al. -- incl., <mark>Yingkang Zhang</mark> |
|*Appeared on*| *2022-10-11*|
|*Comments*| *6 pages, 1 figure, accepted for publication in ApJL*|
|**Abstract**| High-resolution imaging is crucial for exploring the origin and mechanism of radio emission in quasars, especially at high redshifts. We present 1.5 GHz Very Long Baseline Array (VLBA) images of the radio continuum emission from the radio-intermediate quasar (RIQ) J2242+0334 at $z = 5.9$. This object was previously detected at both 1.5 GHz and 3 GHz with the Karl G. Jansky Very Large Array (VLA) as a point source. However, there is no clear detection in the VLBA images at both the full resolution of 10.7 milliarcsecond (mas) $\times$ 4.5 mas (61.7 pc $\times$ 26.0 pc) and a tapered resolution of 26 mas $\times$ 21 mas (150 pc $\times$ 121 pc). This suggests that the radio emission from the quasar is diffuse on mas scales with surface brightness fainter than the $3\sigma$ detection limit of 40.5 $\mu \rm Jy \ beam^{-1}$ in the full resolution image. The radio emission in the RIQ J2242+0334 is likely to be wind-like (i.e., diffuse) rather than in the form of collimated jets. This is different from the previous radio detections of the most luminous quasars at $z \sim$6 which are usually dominated by compact, high brightness temperature radio sources. Meanwhile, compared with RIQs at low redshifts, the case of J2242+0334 suggests that not all RIQs are beamed radio-quiet quasars. This optically faint RIQ provides an important and unique example to investigate the radio activity in the less powerful active galactic nuclei at the earliest cosmic epoch. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03873-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03873) | **The Solar Neighborhood L: Spectroscopic Discovery of K Dwarfs Younger  than 1 Gyr and New Binaries within 30 Parsecs**  |
|| Hodari-Sadiki Hubbard-James, et al. -- incl., <mark>Todd J. Henry</mark> |
|*Appeared on*| *2022-10-11*|
|*Comments*| *Accepted for publication in the Astronomical Journal. this https URL*|
|**Abstract**| As part of a comprehensive effort to characterize the nearest stars, the CHIRON echelle spectrograph on the CTIO/SMARTS 1.5m telescope is being used to acquire high resolution (R = 80000) spectra of K dwarfs within 50 parsecs. This paper provides spectral details about 35 K dwarfs from five benchmark sets with estimated ages spanning 20 Myr -- 5.7 Gyr. Four spectral age and activity indicators are tested, three of which aligned with the estimated ages of the benchmark groups -- the Na I doublet (5889.95 $\r{A}$ and 5895.92 $\r{A}$), the H$\alpha$ line (6562.8 $\r{A}$), and the Li I resonance line (6707.8 $\r{A}$). The benchmark stars are then used to evaluate seven field K dwarfs exhibiting variable radial velocities for which initial CHIRON data did not show obvious companions. Two of these stars are estimated to be younger than 700 Myr, while one exhibits stellar activity unusual for older K dwarf field stars and is possibly young. The four remaining stars turn out to be spectroscopic binaries, two of which are being reported here for the first time with orbital periods found using CHIRON data. Spectral analysis of the combined sample of 42 benchmark and variable radial velocity stars indicates temperatures ranging from 3900--5300 K and metallicities from $-$0.4 $<$ [Fe/H] $<$ $+$0.2. We also determine $log g$ = 4.5--4.7 for main sequence K dwarfs. Ultimately, this study will target several thousand of the nearest K dwarfs, and provide results that will serve present and future studies of stellar astrophysics and exoplanet habitability. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.04316-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.04316) | **In-flight radiometric calibration of the ExoMars TGO Colour and Stereo  Surface Imaging System**  |
|| Antoine Pommerol, et al. -- incl., <mark>Livio L. Tornabene</mark> |
|*Appeared on*| *2022-10-11*|
|*Comments*| **|
|**Abstract**| The Colour and Stereo Surface Science Imaging System (CaSSIS) of the ExoMars Trace Gas Orbiter returns on average twenty images per day of the Martian surface, most of them in 3 or 4 colours and some of them in stereo. CaSSIS uses a push-frame approach to acquire colour images, with four bandpass filters deposited directly above the sensor and an imaging cadence synchronized with the ground track velocity to cover the imaged area with tens of small, partially overlapping images. These "framelets" are later map-projected and mosaicked to build the final image. This approach offers both advantages and challenges in terms of radiometric calibration. While the collection of dark and flatfield frames is considerably enhanced by the frequent and fast acquisition of tens of successive images, mosaics assembled from the adjacent framelets highlight the straylight and changes in the bias of the detector. Both issues have been identified on CaSSIS images, with low intensities overall (up to a few percents), but sufficient to generate prominent artefacts on the final assembled colour images. We have therefore developed methods to correct these artefacts that are now included into the radiometric calibration pipeline. We detail here the different steps of the calibration procedure and the generation of the products used for calibration, and discuss the efficacy of the corrections. The relative uncertainties on the bias and flatfield frames are low, of the order of 0.2 and 0.1 percents, respectively. The uncertainty on the absolute radiometric calibration is of 3 percents, which is quite low for such an instrument. The straylight adds an estimated about 1 percent error to the absolute calibration. The residuals after corrections of the straylight and bias offsets are of the order of a few DNs to tens of DNs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.04721.md
    + _build/html/tmp_2210.04721/./compare_spectra_ug.png
    + _build/html/tmp_2210.04721/./mtype_synth.png
    + _build/html/tmp_2210.04721/./compare_nodes_gir.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Tables A.1--A.3 are fully available in electronic form at theCDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or viahttp://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.04721-b31b1b.svg)](https://arxiv.org/abs/2210.04721)<mark>Appeared on: 2022-10-11</mark> - _15 pages, 15 figures. Accepted by Astronomy & Astrophysics_

</div>
<div id="authors">

E. Franciosini, et al. -- incl., <mark><mark>K. Lind</mark></mark>, <mark><mark>G. Guiglion</mark></mark>, <mark><mark>E.J. Alfaro</mark></mark>, <mark><mark>M. Bergemann</mark></mark>, <mark><mark>U. Heiter</mark></mark>, <mark><mark>J. Lewis</mark></mark>

</div>
<div id="abstract">

**Abstract:** The Gaia-ESO Survey (GES) is a large public spectroscopic survey that wascarried out using the multi-object FLAMES spectrograph at the Very LargeTelescope. The survey provides accurate radial velocities, stellarparameters, and elemental abundances for$\sim$115 000 stars in all MilkyWay components.In this paper we describe the method adopted in the final data release toderive lithium equivalent widths (EWs) and abundances.Lithium EWs were measured using two different approaches for FGK and M-typestars, to account for the intrinsic differences in the spectra. For FGKstars, we fitted the lithium line using Gaussian components, while directintegration over a predefined interval was adopted for M-type stars. Carewas taken to ensure continuity between the two regimes. Abundances werederived using a new set of homogeneous curves of growth that were derivedspecifically for GES, and which were measured on a synthetic spectral gridconsistently with the way the EWs were measured. The derived abundances werevalidated by comparison with those measured by other analysis groups usingdifferent methods.Lithium EWs were measured for$\sim$40 000 stars, and abundances couldbe derived for$\sim$38 000 of them. The vast majority of the measures(80\%) have been obtained for stars in open cluster fields. The remainingobjects are stars in globular clusters, or field stars in the Milky Waydisc, bulge, and halo.The GES dataset of homogeneous lithium abundances described here will bevaluable for our understanding of several processes, from stellar evolutionand internal mixing in stars at different evolutionary stages to Galacticevolution.

</div>

<div id="div_fig1">

<img src="tmp_2210.04721/./compare_spectra_ug.png" alt="Fig1" width="100%"/>

**Figure 1. -** UVES (black, S/N=119) and Giraffe (orange, S/N=223) spectra of the
giant star 07380545+2136507 which was observed with both instruments. The
vertical green and blue dashed lines mark the positions of the lithium
doublet and of the 6707.4 \AA  iron line, respectively. (*fig:compare_ug*)

</div>
<div id="div_fig2">

<img src="tmp_2210.04721/./mtype_synth.png" alt="Fig4" width="100%"/>

**Figure 4. -** Synthetic spectra of M-type dwarf stars with $T_\mathrm{eff}=4000$,
3600 and 3200 K (from top to bottom), $\log g=4.5$ and solar metallicity, at
the Giraffe resolution. Spectra are plotted for $A\mathrm{(Li)}=-1.0$(black
line), 0.0 (red line), $+1.0$(green line), and $+3.0$(blue line). The
vertical dotted line shows the mean position of the lithium doublet. (*fig:mtype_synt*)

</div>
<div id="div_fig3">

<img src="tmp_2210.04721/./compare_nodes_gir.png" alt="Fig8" width="100%"/>

**Figure 8. -** Comparison between the abundances derived by Arcetri with those
derived by other nodes as a function of $T_\mathrm{eff}$, for stars observed
with Giraffe. The plotted differences $\Delta A(\mathrm{Li})$ are computed
as the other node minus Arcetri, and are colour-coded by the Arcetri
abundance. Only stars with detections in both nodes are plotted. The
3-$\sigma$ clipped average and standard deviation of the abundance
differences are indicated in the respective panels. (*fig:comp_gir*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

59  publications in the last 7 days.
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers